In [1]:
import numpy as np
import scipy as sp
from importlib import reload
import neuraltda.topology2 as tp2
import glob
import os
import pickle
import matplotlib.pyplot as plt
import h5py as h5
%matplotlib inline

import seaborn as sns
import pandas as pd
from neuraltda import TPLCP
from neuraltda import FRLCP
reload(TPLCP)

import datetime
daystr = datetime.datetime.now().strftime('%Y%m%d')
figsavepth = '/home/brad/DailyLog/'+daystr+'/'
print(figsavepth)

/home/brad/.conda/envs/neuraltda-4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/home/brad/DailyLog/20181112/


In [22]:
birds = ['B1083','B1056','B1235', 'B1075']

# Block Paths for each bird
bps = {'B1083': '/home/brad/krista/B1083/P03S03/', 'B1075': '/home/brad/krista/B1075/P01S03/',
       'B1235': '/home/brad/krista/B1235/P02S01/', 'B1056': '/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/'}

# Ground Truth Stimuli classes
B1083_classes = {'M_40k': 'R', 'N_40k':'L', 'O_40k':'R', 'P_40k':'L'}
B1075_classes = {'E_40k': 'R', 'F_40k':'L', 'G_40k':'R', 'H_40k':'L'}
B1087_classes = {'M_40k': 'R', 'N_40k':'L', 'O_40k':'R', 'P_40k':'L'}
B1235_classes = {'E_scaled_burung': 'R', 'F_scaled_burung':'L', 'G_scaled_burung':'R', 'H_scaled_burung':'L'}
B1056_classes = {'A_scaled_burung': 'R', 'B_scaled_burung':'L', 'C_scaled_burung':'R', 'D_scaled_burung':'L'}

# Familiar stims for each bird
B1083_stims = ['M_40k', 'N_40k', 'O_40k', 'P_40k']
B1075_stims = ['E_40k', 'F_40k', 'G_40k', 'H_40k']
B1235_stims = ['E_scaled_burung', 'F_scaled_burung', 'G_scaled_burung', 'H_scaled_burung']
B1056_stims = ['A_scaled_burung', 'B_scaled_burung', 'C_scaled_burung', 'D_scaled_burung']

# Bird Dictionaries
stim_classes = {'B1083': B1083_classes, 'B1075': B1075_classes,
                'B1235': B1235_classes, 'B1056': B1056_classes}
bird_stims = {'B1083': B1083_stims, 'B1075': B1075_stims,
              'B1235': B1235_stims, 'B1056': B1056_stims}

B1083_unf_stims = ['I_40k', 'J_40k', 'K_40k', 'L_40k']

B1075_unf_stims = ['A_40k', 'B_40k', 'C_40k', 'D_40k',
                   'I_40k', 'J_40k', 'K_40k', 'L_40k',
                   'M_40k', 'N_40k', 'O_40k', 'P_40k']

B1235_unf_stims = ['A_scaled_burung', 'B_scaled_burung', 'C_scaled_burung', 'D_scaled_burung',
                   'I_scaled_burung', 'J_scaled_burung', 'K_scaled_burung', 'L_scaled_burung',
                   'M_scaled_burung', 'N_scaled_burung', 'O_scaled_burung', 'P_scaled_burung',]

B1056_unf_stims = ['E_scaled_burung', 'F_scaled_burung', 'G_scaled_burung', 'H_scaled_burung',
                   'I_scaled_burung', 'J_scaled_burung', 'K_scaled_burung', 'L_scaled_burung',
                   'M_scaled_burung', 'N_scaled_burung', 'O_scaled_burung', 'P_scaled_burung',]

stimclab = ['R', 'L']
bird_unf_stims = {'B1083': B1083_unf_stims, 'B1075': B1075_unf_stims,
                  'B1235': B1235_unf_stims, 'B1056': B1056_unf_stims}


In [3]:
# Bin the data
# Binning Parameters
winsize = 10.0                # Window size, 10.0 ms is standard for what I've done so far
dtovr = winsize*0.5           # Overlap. I always set to 50% window size
comment = 'ForLogisticPrediction'   # Logistic Prediction
segment_info = [0,0]          # Look only when stimulus is playing

# Bin the data from each bird
bdfs = []
for bird in birds:
    block_path = bps[bird]
    bfdict = tp2.dag_bin(block_path, winsize, segment_info, cluster_group=['Good'], dt_overlap=dtovr, comment=comment)
    bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
    print(bdf)
    bdfs.append([bird, bdf])

/home/brad/krista/B1083/P03S03/binned_data/win-10.0_dtovr-5.0_seg-0-0-ForLogisticPrediction/20171113T203031Z-10.0-5.0.binned
/home/brad/krista/B1056/klusta/phy020516/Pen01_Lft_AP100_ML1300__Site03_Z2500__B1056_cat_P01_S03_1/binned_data/win-10.0_dtovr-5.0_seg-0-0-ForLogisticPrediction/20171113T203059Z-10.0-5.0.binned
/home/brad/krista/B1235/P02S01/binned_data/win-10.0_dtovr-5.0_seg-0-0-ForLogisticPrediction/20171113T203048Z-10.0-5.0.binned
/home/brad/krista/B1075/P01S03/binned_data/win-10.0_dtovr-5.0_seg-0-0-ForLogisticPrediction/20171113T203042Z-10.0-5.0.binned


In [4]:
# Set parameters for logistic prediction analysis
# Betti and Logistic Parameters
# ORIGINAL
reload(tp2)
analysis_id = 'LogisticPredictionBetti'
thresh = 4.0                               # Threshold
nperms = 20                                # 20 permutations of...
ncellsperm = 30                            # 30 cells each
dims = [0,1,2,3,4]                         # Compute the first five betti numbers
Ntimes = 10                                # 10 time points equally spaced.  This makes a 50 dimensional feature vector (5 bettis * 10 time points)
twin = np.linspace(0, 6000, Ntimes)        # Setup time vector
p_test = 0.2                               # Train on 80% of data, test on 20%
N_tests = 60                               # Do sixty independent regressions

In [6]:
# compute betti curves
bcAlls = {}
for b in bdfs:
    bird = b[0]
    bdf = b[1]
    print(bird)
    block_path = bps[bird]
    bc, t, t_ms = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, dims, twin, winsize, dtovr)
    bcAlls[bird] = bc
    

B1083
B1056
B1235
B1075


In [7]:
# Save the output 
with open(os.path.join(figsavepth, 'bcAll_original_parameters.pkl'), 'wb') as f:
    pickle.dump(bcAlls, f)

In [5]:
# Save the output 
with open(os.path.join(figsavepth, 'bcAll_original_parameters.pkl'), 'rb') as f:
    bcAlls = pickle.load(f)

UnsupportedOperation: read

In [ ]:
# Behavioral class decoder accuracy:  Real vs shuffled. 
name = 'BehavioralClass'
acframe_behavioralclass = pd.DataFrame(columns=['Bird', 'Accuracy', 'Shuffle'])
for b in bdfs:
    bird = b[0]
    bdf = b[1]
    stimc = stim_classes[bird]
    stims = bird_stims[bird]
    (bcAll, t, t_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, dims, twin, winsize, dtovr)
 
    accuracies = TPLCP.predict_stimuli_classes(bcAll, Ntimes, stims, stimc, p_test, N_tests)
    accuracies_shuff = TPLCP.predict_stimuli_classes(bcAll, Ntimes, stims, stimc, p_test, N_tests, shuff_Y=True)
    acframeapp = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies, 'Shuffle': N_tests*['No'] })
    acframeapps = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies_shuff, 'Shuffle': N_tests*['Yes'] })
    acframe_behavioralclass = acframe_behavioralclass.append(acframeapp)
    acframe_behavioralclass = acframe_behavioralclass.append(acframeapps)   

In [8]:
# Behavioral class decoder accuracy:  Real vs shuffled. 
name = 'BehavioralClass'
acframe_behavioralclass = pd.DataFrame(columns=['Bird', 'Accuracy', 'Shuffle'])
for b in bdfs:
    bird = b[0]
    bdf = b[1]
    stimc = stim_classes[bird]
    stims = bird_stims[bird]
    print(stims)
    bcAll = bcAlls[bird]
 
    accuracies = TPLCP.predict_stimuli_classes(bcAll, stims, stimc, p_test, N_tests)
    accuracies_shuff = TPLCP.predict_stimuli_classes(bcAll, stims, stimc, p_test, N_tests, shuff_Y=True)
    acframeapp = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies, 'Shuffle': N_tests*['No'] })
    acframeapps = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies_shuff, 'Shuffle': N_tests*['Yes'] })
    acframe_behavioralclass = acframe_behavioralclass.append(acframeapp)
    acframe_behavioralclass = acframe_behavioralclass.append(acframeapps)   

['M_40k', 'N_40k', 'O_40k', 'P_40k']
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
tot

total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts:

In [9]:
# Save the output 
with open(os.path.join(figsavepth, 'acframe_behavioralclass.pkl'), 'wb') as f:
    pickle.dump(acframe_behavioralclass, f)

In [10]:
# Stimulus ID decoder Accuracy
name = 'PredictStimulusID'
acframe_stimulusID = pd.DataFrame(columns=['Bird', 'Accuracy', 'Shuffle'])
for b in bdfs:
    bird = b[0]
    bdf = b[1]
    print(bird)
    #(bcAll, t, t_ms) =tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, dims, twin, winsize, dtovr)
    bcAll = bcAlls[bird]
    print('Completed Betti Computation...')

    stims = list(bcAll.keys())[-8:]
    stimc = {x: x for x in stims}
    accuracies = TPLCP.predict_stimuli_classes(bcAll, stims, stimc, p_test, N_tests)
    accuracies_shuff = TPLCP.predict_stimuli_classes(bcAll, stims, stimc, p_test, N_tests, shuff_Y=True)
    acframeapp = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies, 'Shuffle': N_tests*['No'] })
    acframeapps = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies_shuff, 'Shuffle': N_tests*['Yes'] })
    acframe_stimulusID = acframe_stimulusID.append(acframeapp)
    acframe_stimulusID = acframe_stimulusID.append(acframeapps)

B1083
Completed Betti Computation...
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
tot

total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts: 3200, ntrain: 2560
total pts:

In [11]:
with open(os.path.join(figsavepth, 'acframe_stimulusID_8stims.pkl'), 'wb') as f:
    pickle.dump(acframe_stimulusID, f)

In [24]:
ame = 'PredictArbitraryUnfamiliar'
reload(TPLCP)
acframe_arbunf = pd.DataFrame(columns=['Bird', 'Accuracy', 'Shuffle'])
for b in bdfs:
    bird = b[0]
    bdf = b[1]
    print(bird)
    #stimc = stim_classes[bird]
    stims = bird_unf_stims[bird]
    #stims = bird_stims[bird]
#     (bcAll, t, t_ms) =tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, dims, twin, winsize, dtovr)
#     print('Completed Betti Computation...')

    bcAll = bcAlls[bird]
    #bc2s, ts, ts_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, 2, twin, winsize, dtovr, shuffle=True)
    #bc1s, ts, ts_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, 1, twin, winsize, dtovr, shuffle=True)
    #bc0s, ts, ts_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, 0, twin, winsize, dtovr, shuffle=True)
    #bc3s, ts, ts_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, 3, twin, winsize, dtovr, shuffle=True)
    
    
    #stims = bcAll.keys()
    
    accuracies = TPLCP.predict_arbitrary_classes(bcAll, stims, stimclab, p_test, N_tests)
    accuracies_shuff = TPLCP.predict_arbitrary_classes(bcAll, stims, stimclab, p_test, N_tests, shuff_Y=True)
    acframeapp = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies, 'Shuffle': N_tests*['No'] })
    acframeapps = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies_shuff, 'Shuffle': N_tests*['Yes'] })
    acframe_arbunf = acframe_arbunf.append(acframeapp)
    acframe_arbunf = acframe_arbunf.append(acframeapps)
    
with open(os.path.join(figsavepth, 'acframe_arbunf.pkl'), 'wb') as f:
    pickle.dump(acframe_arbunf, f)

B1083
['I_40k', 'J_40k', 'K_40k', 'L_40k']
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 12

total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts: 4800, ntrain: 3840
total pts:

In [13]:
with open(os.path.join(figsavepth, 'acframe_arbunf.pkl'), 'rb') as f:
    acframe_arbunf = pickle.load(f)

In [14]:
reload(FRLCP)
name = 'FiringRate'
acframe_FR = pd.DataFrame(columns=['Bird', 'Accuracy', 'Shuffle', 'Type'])
for b in bdfs:
    bird = b[0]
    bdf = b[1]
    print(bird)
    poptensors = {}
    with h5.File(bdf, 'r') as f:
        for stim in list(f.keys()):
            poptensors[stim] = tp2.build_permuted_data_tensor(np.array(f[stim]['pop_tens']), ncellsperm, nperms)
    stims = bird_stims[bird]
    stimc = stim_classes[bird]
    accuracies = FRLCP.predict_stimuli_classes_population_FR(poptensors, Ntimes, stims, stimc, p_test, N_tests)
    accuracies_shuff = FRLCP.predict_stimuli_classes_population_FR(poptensors, Ntimes, stims, stimc, p_test, N_tests, shuff_Y=True)
    acframeapp = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies, 'Shuffle': N_tests*['No'], 'Type': N_tests*['FR']})
    acframeapps = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies_shuff, 'Shuffle': N_tests*['Yes'], 'Type': N_tests*['FR']})
    acframe_FR = acframe_FR.append(acframeapp)
    acframe_FR = acframe_FR.append(acframeapps)


with open(os.path.join(figsavepth, 'acframe_FR.pkl'), 'wb') as f:
    pickle.dump(acframe_FR, f)

B1083
20
(30, 1200, 400)
(30, 1200, 400)
(30, 1200, 400)
(30, 1200, 400)
1600
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrai

total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
total pts: 1600, ntrain: 1280
20
(30, 1325, 400)
(30, 1325, 400)
(30, 1325, 400)
(30, 1325, 400)
1600
total pts: 1600, ntrain: 128

In [ ]:
ame = 'PredictArbitraryUnfamiliar'
reload(TPLCP)
acframe_arbunf = pd.DataFrame(columns=['Bird', 'Accuracy', 'Shuffle'])
for b in bdfs:
    bird = b[0]
    bdf = b[1]
    print(bird)
    #stimc = stim_classes[bird]
    stims = bird_unf_stims[bird]
#     (bcAll, t, t_ms) =tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, dims, twin, winsize, dtovr)
#     print('Completed Betti Computation...')

    bcAll = bcAlls[bird]
    #bc2s, ts, ts_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, 2, twin, winsize, dtovr, shuffle=True)
    #bc1s, ts, ts_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, 1, twin, winsize, dtovr, shuffle=True)
    #bc0s, ts, ts_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, 0, twin, winsize, dtovr, shuffle=True)
    #bc3s, ts, ts_ms) = tp2.compute_betti_curves(analysis_id, block_path, bdf, thresh, nperms, ncellsperm, 3, twin, winsize, dtovr, shuffle=True)
    
    
    #stims = bcAll.keys()
    stimc = {'M_scaled_burung': 'R', 'N_scaled_burung': 'L', 'O_scaled_burung': 'R', 'P_scaled_burung': 'L', 'M_40k': 'R', 'N_40k': 'L', 'O_40k':'R', 'P_40k': 'L', 'I_40k': 'R', 'J_40k': 'L', 'K_40k': 'R', 'L_40k': 'L'}
    accuracies = TPLCP.predict_stimuli_classes(bcAll, stims, stimc, p_test, N_tests)
    accuracies_shuff = TPLCP.predict_stimuli_classes(bcAll, stims, stimc, p_test, N_tests, shuff_Y=True)
    acframeapp = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies, 'Shuffle': N_tests*['No'] })
    acframeapps = pd.DataFrame({'Bird': N_tests*[bird], 'Accuracy': accuracies_shuff, 'Shuffle': N_tests*['Yes'] })
    acframe_arbunf = acframe_arbunf.append(acframeapp)
    acframe_arbunf = acframe_arbunf.append(acframeapps)
    
with open(os.path.join(figsavepth, 'acframe_arbunf.pkl'), 'wb') as f:
    pickle.dump(acframe_arbunf, f)

In [21]:
bcAlls['B1235']['B_scaled_burung'].shape

(5, 10, 400)